# AutoAI Modeling Notebook

##### This notebook is generated from AutoAI after and AutoAI experiment is run. Once feature engineering is completed run the following steps to generate this notebook.
•	Within the project created, click on "Add to project", select "AutoAI experiment"  
•	Enter a name & description and click "Create"  
•	In the Add data source screen, select "Select from project" and select the df_training.csv asset  
•	In the Select prediction column section, click on the column name "default" and click on the "Experiment Settings" option  
•	In "Experiment Setting" > "Data Source" section,   
      i. Set "Training data split" to 90% (hold out data split - 10%) This can be modified in this notebook in "2.Compose Pipeline" section with _input_metadata  
      ii. In "Select Columns to Include", uncheck the id columns (loan_id, branch_id) and "default_12mth" column. You can also uncheck any other column you don't want to use. This can be modified in this notebook in "2.Compose Pipeline" section with "column_selector".  
•	In "Experiment Setting" > "Prediction" section,  
      i. Ensure, "Binary Classification" is selected for "Prediction Type"  
      ii. Ensure "Positive Class" is set to "True"  
      iii. Select "F1" for "Optimized Metric". This can be modified in this notebook in "2.Compose Pipeline" section with _input_metadata. Other values you can set are Accuracy, ROC AUC, Precision, Recall and Log Loss.  
      iv. In "Algorithms to test", ensure all the algorithms are selected (for the first run with all the features, it is better to choose all the features to feature importance, which can be used to select features for model tuning)  
      v. In "Algorithms to use", You can select number of different algorithms you want to compare in the end. Each algorithm produces 4 pipelines ranging for base feature models to hyper parameter optimized models. Ideally, you would want to select 2 for this section.  
•	Click on the "Save Setting" and the "Run Experiment" option to begin  
•	When the experiment is complete, click on the top model from the leaderboard that you want to use and click on the "Save Pipeline as Notebook". This command will generate this notebook.  
•	Alternatively, you can save the model when the experiment is complete. Click on the top model from the leaderboard that you want to use and click on the "Save as Model". We will discuss how to deploy the model at a later stage.  

################################################################################
#Licensed Materials - Property of IBM
#(C) Copyright IBM Corp. 2020
#US Government Users Restricted Rights - Use, duplication disclosure restricted
#by GSA ADP Schedule Contract with IBM Corp.
################################################################################

The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below.
Specifically, the Source Components and Sample Materials clause included in the License Information document for
Watson Studio Auto-generated Notebook applies to the auto-generated notebooks. 
By downloading, copying, accessing, or otherwise using the materials, you agree to the License Terms.
http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1

## IBM AutoAI Auto-Generated Notebook v1.11.10
**Note**: Notebook code generated using AutoAI will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
This pipeline is optimized for the original dataset.The pipeline may fail or produce sub-optimium results if used with different data.
For different data, please consider returning to AutoAI Experiments to generate a new pipeline.
Please read our documentation for more information: 
(Cloud Platform) https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html .
(Cloud Pak For Data) https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/autoai-notebook.html .

Before modifying the pipeline or trying to re-fit the pipeline, consider: 
The notebook converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).
The known_values_list is passed by reference and populated with categorical values during fit of the preprocessing pipeline.Delete its members before re-fitting.

### Representing Pipeline from run: Pipeline_1 from run 98767f4e4cb44e35b07043c9501d38e9
AutoAI produces 4 pipelines and ranks them based on the performance metric you select for each of the output algorithm selected.

### 1. Set Up

In [2]:

try:
    import autoai_libs
except Exception as e:
    import subprocess
    out = subprocess.check_output('pip install autoai-libs'.split(' '))
    for line in out.splitlines():
        print(line)
    import autoai_libs
import sklearn
try:
    import xgboost
except:
    print('xgboost, if needed, will be installed and imported later')
try:
    import lightgbm
except:
    print('lightgbm, if needed, will be installed and imported later')
from sklearn.cluster import FeatureAgglomeration
import numpy
from numpy import inf, nan, dtype, mean
from autoai_libs.sklearn.custom_scorers import CustomScorers
import sklearn.ensemble
from autoai_libs.cognito.transforms.transform_utils import TExtras, FC
from autoai_libs.transformers.exportable import *
from autoai_libs.utils.exportable_utils import *
from sklearn.pipeline import Pipeline
known_values_list=[]

b'Collecting autoai-libs'
b'  Downloading autoai_libs-1.10.12-37-cp36-cp36m-manylinux1_x86_64.whl (4.3 MB)'
b'Collecting scikit-learn==0.20.3'
b'  Downloading scikit_learn-0.20.3-cp36-cp36m-manylinux1_x86_64.whl (5.4 MB)'
b'Requirement already satisfied: numpy>=1.16.4 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from autoai-libs) (1.16.6)'
b'Requirement already satisfied: pandas<1.0.0,>=0.24.2 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from autoai-libs) (0.24.2)'
b'Collecting category-encoders==2.1.0'
b'  Downloading category_encoders-2.1.0-py2.py3-none-any.whl (100 kB)'
b'Requirement already satisfied: scipy>=0.13.3 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from scikit-learn==0.20.3->autoai-libs) (1.3.2)'
b'Requirement already satisfied: python-dateutil>=2.5.0 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from pandas<1.0.0,>=0.24.2->autoai-libs) (2.7.5)'
b'Requirement already satisfied: pytz>=2011k in 

In [3]:
# compose a decorator to assist pipeline instantiation via import of modules and installation of packages
def decorator_retries(func):
    def install_import_retry(*args, **kwargs):
        retries = 0
        successful = False
        failed_retries = 0
        while retries < 100 and failed_retries < 10 and not successful:
            retries += 1
            failed_retries += 1
            try:
                result = func(*args, **kwargs)
                successful = True
            except Exception as e:
                estr = str(e)
                if estr.startswith('name ') and estr.endswith(' is not defined'):
                    try:
                        import importlib
                        module_name = estr.split("'")[1]
                        module = importlib.import_module(module_name)
                        globals().update({module_name: module})
                        print('import successful for ' + module_name)
                        failed_retries -= 1
                    except Exception as import_failure:
                        print('import of ' + module_name + ' failed with: ' + str(import_failure))
                        import subprocess
                        print('attempting pip install of ' + module_name)
                        process = subprocess.Popen('pip install ' + module_name, shell=True)
                        process.wait()
                        try:
                            print('re-attempting import of ' + module_name)
                            module = importlib.import_module(module_name)
                            globals().update({module_name: module})
                            print('import successful for ' + module_name)
                            failed_retries -= 1
                        except Exception as import_or_installation_failure:
                            print('failure installing and/or importing ' + module_name + ' error was: ' + str(
                                import_or_installation_failure))
                            raise (ModuleNotFoundError('Missing package in environment for ' + module_name +
                                                       '? Try import and/or pip install manually?'))
                elif type(e) is AttributeError:
                    if 'module ' in estr and ' has no attribute ' in estr:
                        pieces = estr.split("'")
                        if len(pieces) == 5:
                            try:
                                import importlib
                                print('re-attempting import of ' + pieces[3] + ' from ' + pieces[1])
                                module = importlib.import_module('.' + pieces[3], pieces[1])
                                failed_retries -= 1
                            except:
                                print('failed attempt to import ' + pieces[3])
                                raise (e)
                        else:
                            raise (e)
                else:
                    raise (e)
        if successful:
            print('Pipeline successfully instantiated')
        else:
            raise (ModuleNotFoundError(
                'Remaining missing imports/packages in environment? Retry cell and/or try pip install manually?'))
        return result
    return install_import_retry


### 2. Compose Pipeline

#### Section 1: This section details the steps of AutoAI Experiment configuration - Experiment Settings > Data Source

#### Understanding the "_input_metadata"
input meta data lets you modify evaluation criteria for you model

_input_metadata = {
                   <p>'target_label_name': 'default', #Incase the name of your target variable changes, this is the parameter that needs to be modified <br>
                   <p>'learning_type': 'classification', #This parameter can either be regression or classification. Clustering is not supported by AutoAI at this point <br>
                   <p>'run_uid': 'f6d260e483ad4c4e82dc8b5b29f856c9', #This parameter is the UID of the AutoAI run, which is generated and is specific to the experiment that was run for this pipeline <br>
                   <p>'pn': 'P1', <br>
                   <p>'cv_num_folds': 3, #On AutoAI number of cross validation folds is constant at 3 and cannot be changed, however this value can be modified here with this parameter. <br>
                   <p>'holdout_fraction': 0.1, #On AutoAI training:holdout data is capped between 95-85:5-15 %. However this can be abjusted beyond these limits with this parameter. <br>
                   <p>'optimization_metric': 'f1', #This is the permance metric that was selected for evaluation the model for this pipeline. <br>
                   <p>'pos_label': None, <br>
                   <p>'random_state': 33, #This parameter modifies how the data is split for cross validation. <br>
                   <p>'data_source': ''} #If datasources, different from the project data assest mentioned in credentials->'ASSET_ID' are used for modelling, they need to be mentioned here <br>

In [4]:
# metadata necessary to replicate AutoAI scores with the pipeline
_input_metadata = {'target_label_name': 'default_12mth', 'learning_type': 'classification', 'run_uid': '98767f4e4cb44e35b07043c9501d38e9', 'pn': 'P1', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'f1', 'pos_label': None, 'random_state': 33, 'data_source': ''}

# define a function to compose the pipeline, and invoke it
@decorator_retries
def compose_pipeline():
    import numpy
    from numpy import nan, dtype, mean
    #
    # composing steps for toplevel Pipeline
    #
    _input_metadata = {'target_label_name': 'default_12mth', 'learning_type': 'classification', 'run_uid': '98767f4e4cb44e35b07043c9501d38e9', 'pn': 'P1', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'f1', 'pos_label': None, 'random_state': 33, 'data_source': ''}
    steps = []
    #
    # This step ensures you can input your training data with all the extra columns and filter out required columns with column selector.
    # You will have to maintain the same input order for your test set
    # However, if you do not wish to drop columns, or want to use the training data frame as is, ensure to set activate_flag to False
    # If your feature column order changes, or you wish to drop certain features from your training set, use the array of columns_indices_list to do so.
    steps.append(('column_selector', autoai_libs.transformers.exportable.ColumnSelector(activate_flag=True, columns_indices_list=[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54])))
    #
    # composing steps for preprocessor Pipeline
    #
    preprocessor__input_metadata = None
    preprocessor_steps = []
    #
    # composing steps for preprocessor_features FeatureUnion
    #
    preprocessor_features_transformer_list = []
    #
    # composing steps for preprocessor_features_categorical Pipeline
    # If your column order change, this section will need to be reordered. 
    # Alternatively, you can Impute and encode your categorical variables outside the pipeline and just use preprocessor_features_numeric Pipeline
    #
    preprocessor_features_categorical__input_metadata = None
    preprocessor_features_categorical_steps = []
    preprocessor_features_categorical_steps.append(('cat_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[2, 3, 4, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])))
    preprocessor_features_categorical_steps.append(('cat_compress_strings', autoai_libs.transformers.exportable.CompressStrings(activate_flag=True, compress_type='hash', dtypes_list=['float_int_num', 'char_str', 'char_str', 'float_int_num', 'float_int_num', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'float_int_num', 'float_int_num', 'float_int_num', 'float_int_num'], missing_values_reference_list=['', '-', '?', nan], misslist_list=[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []])))
    preprocessor_features_categorical_steps.append(('cat_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[])))
    preprocessor_features_categorical_steps.append(('cat_unknown_replacer', autoai_libs.transformers.exportable.NumpyReplaceUnknownValues(filling_values=nan, filling_values_list=[100001, nan, nan, 100001, 100001, nan, nan, nan, nan, nan, nan, nan, 100001, 100001, 100001, 100001], known_values_list=[[12.0, 24.0, 36.0, 48.0, 60.0], [306851216158335538240511469114392712665, 284391617279283623785344792025841180603, 21820224760534408439945813044982358529, 241827752477821275412298589787543017525], [188232129152488152603460248363708042922, 116716425681947542349874901877587682272], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0], [180108213698688811818628024229721526293, 144562992748704032427581965011549348079, 236051811100429432872591611432277919360, 156972773494665108799270924439769861959], [11765317584926780588481940266164704654, 261960909976821411497529214859857514665, 95302490137963064818839228327672545332, 325230222422057573982771959716439826699, 180113569312430956460304045157917032883, 268028982883966790315113293623602377941, 172717344712727789242215339191801400999, 250163963194804354786921628306351307985, 300395095876485387403334997981032045437], [22827317255583933288434452265241377565, 300590053663008657466195249252801950921, 196241008562390614726385535674331126248], [188232129152488152603460248363708042922, 116716425681947542349874901877587682272], [295097738706392148777174281837800685781, 284237704998450748001787865733726814702, 272573425929578095555179911497340952271, 48017888444116824979724518598419382334], [188232129152488152603460248363708042922, 116716425681947542349874901877587682272], [188232129152488152603460248363708042922, 116716425681947542349874901877587682272], [14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0], [8.0, 9.0, 10.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0]], missing_values_reference_list=['', '-', '?', nan])))
    preprocessor_features_categorical_steps.append(('boolean2float_transformer', autoai_libs.transformers.exportable.boolean2float(activate_flag=True)))
    preprocessor_features_categorical_steps.append(('cat_imputer', autoai_libs.transformers.exportable.CatImputer(activate_flag=True, missing_values=nan, sklearn_version_family='20', strategy='most_frequent')))
    preprocessor_features_categorical_steps.append(('cat_encoder', autoai_libs.transformers.exportable.CatEncoder(activate_flag=True, categories='auto', dtype=numpy.float64, encoding='ordinal', handle_unknown='error', sklearn_version_family='20')))
    preprocessor_features_categorical_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_categorical_ Pipeline
    preprocessor_features_categorical_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_categorical_steps)
    preprocessor_features_transformer_list.append(('categorical', preprocessor_features_categorical_pipeline))
    #
    # composing steps for preprocessor_features_numeric Pipeline
    #
    preprocessor_features_numeric__input_metadata = None
    preprocessor_features_numeric_steps = []
    preprocessor_features_numeric_steps.append(('num_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[0, 1, 5, 6, 9, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])))
    preprocessor_features_numeric_steps.append(('num_floatstr2float_transformer', autoai_libs.transformers.exportable.FloatStr2Float(activate_flag=True, dtypes_list=['float_num', 'float_int_num', 'float_num', 'float_num', 'float_int_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num', 'float_num'], missing_values_reference_list=[])))
    preprocessor_features_numeric_steps.append(('num_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[])))
    preprocessor_features_numeric_steps.append(('num_imputer', autoai_libs.transformers.exportable.NumImputer(activate_flag=True, missing_values=nan, strategy='median')))
    preprocessor_features_numeric_steps.append(('num_scaler', autoai_libs.transformers.exportable.OptStandardScaler(num_scaler_copy=None, num_scaler_with_mean=None, num_scaler_with_std=None, use_scaler_flag=False)))
    preprocessor_features_numeric_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_numeric_ Pipeline
    preprocessor_features_numeric_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_numeric_steps)
    preprocessor_features_transformer_list.append(('numeric', preprocessor_features_numeric_pipeline))
    # assembling preprocessor_features_ FeatureUnion
    preprocessor_features_pipeline = sklearn.pipeline.FeatureUnion(transformer_list=preprocessor_features_transformer_list)
    preprocessor_steps.append(('features', preprocessor_features_pipeline))
    preprocessor_steps.append(('permuter', autoai_libs.transformers.exportable.NumpyPermuteArray(axis=0, permutation_indices=[2, 3, 4, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 0, 1, 5, 6, 9, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])))
    # assembling preprocessor_ Pipeline
    preprocessor_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_steps)
    steps.append(('preprocessor', preprocessor_pipeline))
    steps.append(('estimator', lightgbm.sklearn.LGBMClassifier(boosting_type='gbdt', class_weight='balanced', colsample_bytree=1.0, importance_type='split', learning_rate=0.1, max_depth=-1, min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0, n_estimators=100, n_jobs=4, num_leaves=31, objective=None, random_state=33, reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0, subsample_for_bin=200000, subsample_freq=0)))
    # assembling  Pipeline
    pipeline = sklearn.pipeline.Pipeline(steps=steps)
    return pipeline
pipeline = compose_pipeline()


/opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


import of lightgbm failed with: No module named 'lightgbm'
attempting pip install of lightgbm
re-attempting import of lightgbm
import successful for lightgbm
Pipeline successfully instantiated


/opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


### 3. Extract needed parameter values from AutoAI run metadata

In [5]:

# Metadata used in retrieving data and computing metrics.  Customize as necessary for your environment.
#data_source='replace_with_path_and_csv_filename'
target_label_name = _input_metadata['target_label_name']
learning_type = _input_metadata['learning_type']
optimization_metric = _input_metadata['optimization_metric']
random_state = _input_metadata['random_state']
cv_num_folds = _input_metadata['cv_num_folds']
holdout_fraction = _input_metadata['holdout_fraction']
if 'data_provenance' in _input_metadata:
    data_provenance = _input_metadata['data_provenance']
else:
    data_provenance = None
if 'pos_label' in _input_metadata and learning_type == 'classification':
    pos_label = _input_metadata['pos_label']
else:
    pos_label = None


### 4. Create dataframe from dataset in Cloud Object Storage

In [6]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_0 = {
    'ENDPOINT': 'unknown cloud service endpoint',
    'IBM_AUTH_ENDPOINT': 'unknown auth_endpoint',
    'APIKEY': 'unspecified api_key for bucket',
    'BUCKET': 'unspecified bucket name',
    'FILE': 'unspecified object name',
    'SERVICE_NAME': 'data_asset',
    'ASSET_ID': 'df_training.csv',
    }


In [7]:
#  Read the data as a dataframe
import pandas as pd

csv_encodings=['UTF-8','Latin-1'] # supplement list of encodings as necessary for your data
df = None
readable = None  # if automatic detection fails, you can supply a filename here

# First, obtain a readable object
# Cloud Object Storage data access
# Assumes COS credentials are in a dictionary named 'credentials_0'
   
credentials = df = globals().get('credentials_0')       
if readable is None and credentials is not None :
    try:
        import types
        import pandas as pd
        import io
        import os
    except Exception as import_exception:
        print('Error with importing packages - check if you installed them on your environment')
    try:
        if credentials['SERVICE_NAME'] == 's3':
            try:
                from botocore.client import Config
                import ibm_boto3
            except Exception as import_exception:
                print('Installing required packages!')
                !pip install ibm-cos-sdk
                print('accessing data via Cloud Object Storage')
            try:
                cos_client = ibm_boto3.resource(service_name=credentials['SERVICE_NAME'],
                                                ibm_api_key_id=credentials['APIKEY'],
                                                ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                                                config=Config(signature_version='oauth'),
                                                endpoint_url=credentials['ENDPOINT'])
            except Exception as cos_exception:
                print('unable to create client for cloud object storage')
            try:
                cos_client.meta.client.download_file(Bucket=credentials['BUCKET'], Filename=credentials['FILE'], Key=credentials['FILE'])
            except Exception as cos_access_exception:
                print('unable to access data object in cloud object storage with credentials supplied') 
            try:
                df = pd.read_csv(credentials['FILE'])
                os.remove(credentials['FILE'])
            except Exception as cos_object_read_exception:
                print('unable to access data object from cos object with path supplied') 
        elif credentials['SERVICE_NAME'] == 'fs':
            print('accessing data via File System')
            try:
                df = pd.read_csv(credentials['FILE'])
            except Exception as FS_access_exception:
                print('unable to access data object in File System with path supplied') 
    except Exception as data_access_exception:
        print('unable to access data object with credentials supplied') 

# IBM Cloud Pak for Data data access
project_filename = globals().get('project_filename')       
if readable is None and 'credentials_0' in globals() and 'ASSET_ID' in credentials_0:
    project_filename = credentials_0['ASSET_ID']
if project_filename is not None:
    print('attempting project_lib access to ' + str(project_filename))
    try:
        from project_lib import Project
        project = Project.access()
        storage_credentials = project.get_storage_metadata()
        readable = project.get_file(project_filename)
    except Exception as project_exception:
        print('unable to access data using the project_lib interface and filename supplied')

        
# Use data_provenance as filename if other access mechanisms are unsuccessful
if readable is None and type(data_provenance) is str:
    print('attempting to access local file using path and name ' + data_provenance)
    readable = data_provenance

# Second, use pd.read_csv to read object, iterating over list of csv_encodings until successful
if readable is not None:
    for encoding in csv_encodings:
        try:
            df = pd.read_csv(readable, encoding=encoding)
            print('successfully loaded dataframe using encoding = ' + str(encoding))
            break
        except Exception as exception_csv:
            print('unable to read csv using encoding ' + str(encoding))
            print('handled error was ' + str(exception_csv))
    if df is None:
        print('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings))
        print(f'Please use \'insert to code\' on data panel to load dataframe.')
        raise(ValueError('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings)))

if isinstance(df,pd.DataFrame):
    print('Data loaded succesfully')


attempting project_lib access to df_training.csv
successfully loaded dataframe using encoding = UTF-8
Data loaded succesfully


### 5. Preprocess Data

In [8]:
# Drop rows whose target is not defined
target = target_label_name # your target name here
if learning_type == 'regression':
    df[target] = pd.to_numeric(df[target], errors='coerce')
df.dropna('rows', how='any', subset=[target], inplace=True)


In [9]:
# extract X and y
df_X = df.drop(columns=[target])
df_y = df[target]


In [10]:
# Detach preprocessing pipeline (which needs to see all training data)
preprocessor_index = -1
preprocessing_steps = [] 
for i, step in enumerate(pipeline.steps):
    preprocessing_steps.append(step)
    if step[0]=='preprocessor':
        preprocessor_index = i
        break
#if len(pipeline.steps) > preprocessor_index+1 and pipeline.steps[preprocessor_index + 1][0] == 'cognito':
    #preprocessor_index += 1
    #preprocessing_steps.append(pipeline.steps[preprocessor_index])
if preprocessor_index >= 0:
    preprocessing_pipeline = Pipeline(memory=pipeline.memory, steps=preprocessing_steps)
    pipeline = Pipeline(steps=pipeline.steps[preprocessor_index+1:])

In [11]:
# Preprocess X
# preprocessor should see all data for cross_validate on the remaining steps to match autoai scores
known_values_list.clear()  #  known_values_list is filled in by the preprocessing_pipeline if needed
preprocessing_pipeline.fit(df_X.values, df_y.values)
X_prep = preprocessing_pipeline.transform(df_X.values)

### 6. Split data into Training and Holdout sets

In [12]:
# determine learning_type and perform holdout split (stratify conditionally)
if learning_type is None:
    # When the problem type is not available in the metadata, use the sklearn type_of_target to determine whether to stratify the holdout split
    # Caution:  This can mis-classify regression targets that can be expressed as integers as multiclass, in which case manually override the learning_type
    from sklearn.utils.multiclass import type_of_target
    if type_of_target(df_y.values) in ['multiclass', 'binary']:
        learning_type = 'classification'
    else:
        learning_type = 'regression'
    print('learning_type determined by type_of_target as:',learning_type)
else:
    print('learning_type specified as:',learning_type)
    
from sklearn.model_selection import train_test_split
if learning_type == 'classification':
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state, stratify=df_y.values)
else:
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state)


learning_type specified as: classification


#### 7. Generate features via Feature Engineering pipeline

In [13]:
#Detach Feature Engineering pipeline if next, fit it, and transform the training data
fe_pipeline = None
if pipeline.steps[0][0] == 'cognito':
    try:
        fe_pipeline = Pipeline(steps=[pipeline.steps[0]])
        X = fe_pipeline.fit_transform(X, y)
        X_holdout = fe_pipeline.transform(X_holdout)
        pipeline.steps = pipeline.steps[1:]
    except IndexError:
        try:
            print('Trying to compose pipeline with some of cognito steps')
            fe_pipeline = Pipeline(steps = list([pipeline.steps[0][1].steps[0],pipeline.steps[0][1].steps[1]]))
            X = fe_pipeline.fit_transform(X, y)
            X_holdout = fe_pipeline.transform(X_holdout)
            pipeline.steps = pipeline.steps[1:]
        except IndexError:
            print('Composing pipeline without cognito steps!')
            pipeline.steps = pipeline.steps[1:]


 ### 8. Additional setup: Define a function that returns a scorer for the target's positive label

In [14]:
# create a function to produce a scorer for a given positive label
def make_pos_label_scorer(scorer, pos_label):
    kwargs = {'pos_label':pos_label}
    for prop in ['needs_proba', 'needs_threshold']:
        if prop+'=True' in scorer._factory_args():
            kwargs[prop] = True
    if scorer._sign == -1:
        kwargs['greater_is_better'] = False
    from sklearn.metrics import make_scorer
    scorer=make_scorer(scorer._score_func, **kwargs)
    return scorer

### 9. Fit pipeline, predict on Holdout set, calculate score, perform cross-validation

In [15]:
# fit the remainder of the pipeline on the training data
pipeline.fit(X,y)

Pipeline(memory=None,
     steps=[('estimator', LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
        colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=4, num_leaves=31,
        objective=None, random_state=33, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0))])

In [16]:
# predict on the holdout data
y_pred = pipeline.predict(X_holdout)

In [17]:
# compute score for the optimization metric
# scorer may need pos_label, but not all scorers take pos_label parameter
from sklearn.metrics import get_scorer
scorer = get_scorer(optimization_metric)
score = None
#score = scorer(pipeline, X_holdout, y_holdout)  # this would suffice for simple cases
pos_label = None  # if you want to supply the pos_label, specify it here
if pos_label is None and 'pos_label' in _input_metadata:
    pos_label=_input_metadata['pos_label']
try:
    score = scorer(pipeline, X_holdout, y_holdout)
except Exception as e1:
    if pos_label is None or str(pos_label)=='':
        print('You may have to provide a value for pos_label in order for a score to be calculated.')
        raise(e1)
    else:
        exception_string=str(e1)
        if 'pos_label' in exception_string:
            try:
                scorer = make_pos_label_scorer(scorer, pos_label=pos_label)
                score = scorer(pipeline, X_holdout, y_holdout)
                print('Retry was successful with pos_label supplied to scorer')
            except Exception as e2:
                print('Initial attempt to use scorer failed.  Exception was:')
                print(e1)
                print('')
                print('Retry with pos_label failed.  Exception was:')
                print(e2)
        else:
            raise(e1)

if score is not None:
    print(score)

0.9600000000000001


In [18]:
# cross_validate pipeline using training data
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, KFold
if learning_type == 'classification':
    fold_generator = StratifiedKFold(n_splits=cv_num_folds, random_state=random_state)
else:
    fold_generator = KFold(n_splits=cv_num_folds, random_state=random_state)
cv_results = cross_validate(pipeline, X, y, cv=fold_generator, scoring={optimization_metric:scorer}, return_train_score=True)
import numpy as np
np.mean(cv_results['test_' + optimization_metric])

0.9256898923232028

In [19]:
cv_results

{'fit_time': array([0.76564598, 0.78127241, 0.78704858]),
 'score_time': array([0.00791049, 0.00759602, 0.00804472]),
 'test_f1': array([0.89473684, 0.94039735, 0.94193548]),
 'train_f1': array([1., 1., 1.])}

In [20]:
#This is the final model that was selected by AutoAI and that can be deployed to WML
final_model = pipeline.fit(X,y)

### 10. Deploy Model and Validate Model Performace (Not generated by AutoAI model notebook)

In [21]:
#Save the model
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import os
token = os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
    "token": token,
    "instance_id" : "openshift",
    "url": "https://dse-cp4d301-cluster2.cpolab.ibm.com//",
    "version": "3.0.0"
}

In [22]:
# access the deployment space
client = WatsonMachineLearningAPIClient(wml_credentials)
client.spaces.list()

------------------------------------  -------------------------  ------------------------
GUID                                  NAME                       CREATED
97daded8-b470-4f45-b646-174dfc7f10d6  loan_default_deploy_space  2020-07-15T16:48:49.767Z
------------------------------------  -------------------------  ------------------------


In [23]:
# You need to set the client space - the deployment space, before deploying the model
#This is the GUID from the result of the previous cell
client.set.default_space("97daded8-b470-4f45-b646-174dfc7f10d6")

'SUCCESS'

In [24]:
# save the model with the proper type/runtime from the list above
model_metadata = {
    client.repository.ModelMetaNames.NAME: "AutoAI_LGBM",
    client.repository.ModelMetaNames.DESCRIPTION: "AutoAI LGBM Model within SKlearn pipeline",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20",
    client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3.6"

}
model_details = client.repository.store_model(model = final_model, meta_props = model_metadata)
model_details

{'metadata': {'name': 'AutoAI_LGBM',
  'guid': 'a5e68542-257c-4548-9331-455f34521b95',
  'description': 'AutoAI LGBM Model within SKlearn pipeline',
  'id': 'a5e68542-257c-4548-9331-455f34521b95',
  'modified_at': '2020-07-21T16:49:25.002Z',
  'created_at': '2020-07-21T16:49:24.002Z',
  'owner': '1000331023',
  'href': '/v4/models/a5e68542-257c-4548-9331-455f34521b95?space_id=97daded8-b470-4f45-b646-174dfc7f10d6',
  'space_id': '97daded8-b470-4f45-b646-174dfc7f10d6'},
 'entity': {'name': 'AutoAI_LGBM',
  'description': 'AutoAI LGBM Model within SKlearn pipeline',
  'content_status': {'state': 'persisted'},
  'space': {'id': '97daded8-b470-4f45-b646-174dfc7f10d6',
   'href': '/v4/spaces/97daded8-b470-4f45-b646-174dfc7f10d6'},
  'type': 'scikit-learn_0.20',
  'runtime': {'id': 'scikit-learn_0.20-py3.6',
   'href': '/v4/runtimes/scikit-learn_0.20-py3.6'}}}

In [25]:
#Check if your model is saved
client.repository.list_models()

------------------------------------  ------------------  ------------------------  -----------------
GUID                                  NAME                CREATED                   TYPE
a5e68542-257c-4548-9331-455f34521b95  AutoAI_LGBM         2020-07-21T16:49:24.002Z  scikit-learn_0.20
06b307ac-bd45-4c0b-9815-8131a99f2567  loan_default_model  2020-07-20T14:06:19.002Z  scikit-learn_0.22
882ed3ba-ba94-4d7c-b7e9-796c0b79e1ce  loan_default_model  2020-07-15T16:49:52.002Z  scikit-learn_0.22
------------------------------------  ------------------  ------------------------  -----------------
